In [1]:
import os
import xarray as xr
import pandas as pd
import numpy as np
import pandas as pd
from scipy.interpolate import RegularGridInterpolator

In [2]:
df0=pd.read_csv(r"E:\\NE_glb_cities_era5_1020.csv")
df0.head()

,Unnamed: 0.1,Unnamed: 0,latitude,longitude,utc_time,xco2,baseXCO2,NE,local_time,year,...,soil_temperature_level_1,surface_pressure,leaf_area_index_high_vegetation,leaf_area_index_low_vegetation,volumetric_soil_water_layer_1surface_net_solar_radiation_hourly,surface_net_thermal_radiation_hourly,volumetric_soil_water_layer_1,surface_net_solar_radiation_hourly,uv10m,LAI
0,0,999250,34.164528,-118.48089,2023-02-06 18:00:11.725021952,421.83026,423.435196,-1.604935,2023-02-06 10:00:11.725021952-08:00,2023,...,284.577347,98499.660156,4.792358,1.024170,NaN,-367060.0,0.282516,1462010.0,2.731091,5.816528
1,1,999251,34.170770,-118.49996,2023-02-06 18:00:11.752018432,421.61475,423.435949,-1.821203,2023-02-06 10:00:11.752018432-08:00,2023,...,284.577347,98499.660156,4.792358,1.024170,NaN,-367060.0,0.282516,1462010.0,2.731091,5.816528
2,2,999252,34.140636,-118.34925,2023-02-06 18:00:11.896012800,421.81088,423.432313,-1.621430,2023-02-06 10:00:11.896012800-08:00,2023,...,285.317581,100632.660156,4.867798,1.668091,NaN,-379043.0,0.270538,1491254.0,2.343180,6.535889
3,3,999253,34.147100,-118.36807,2023-02-06 18:00:11.923009280,421.96448,423.433093,-1.468615,2023-02-06 10:00:11.923009280-08:00,2023,...,285.407425,100716.660156,4.254028,0.794189,NaN,-376133.0,0.275803,1509558.0,2.847821,5.048218
4,4,999254,34.153610,-118.38710,2023-02-06 18:00:11.950005504,422.16742,423.433878,-1.266459,2023-02-06 10:00:11.950005504-08:00,2023,...,285.358597,99137.660156,5.000488,0.993042,NaN,-374100.0,0.267105,1470882.0,2.783441,5.993530


In [5]:
df0.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'latitude', 'longitude', 'utc_time',
       'xco2', 'baseXCO2', 'NE', 'local_time', 'year', 'month', 'hour',
       'geometry', 'index_right', 'COUNTRY', 'NAME_1', 'NAME_2',
       'local_time_str', 'u_component_of_wind_10m', 'v_component_of_wind_10m',
       'dewpoint_temperature_2m', 'temperature_2m', 'soil_temperature_level_1',
       'surface_pressure', 'leaf_area_index_high_vegetation',
       'leaf_area_index_low_vegetation',
       'volumetric_soil_water_layer_1surface_net_solar_radiation_hourly',
       'surface_net_thermal_radiation_hourly', 'volumetric_soil_water_layer_1',
       'surface_net_solar_radiation_hourly', 'uv10m', 'LAI'],
      dtype='object')

In [3]:
import os
import pandas as pd
import xarray as xr
import numpy as np

# 假设你已有的 df0 数据集
# 包含 'latitude', 'longitude', 'datetime' 列
df0['emission'] = np.nan  # 新增 emission 列，先初始化为空值

# 定义文件目录和文件模式
base_dir = r"E:\\GRACED_19082311"
file_pattern = "CarbonMonitor_total_y{year}_m{month:02d}.nc"

# 将 df0 的 latitude 和 longitude 归到最近的 0.05, 0.15, 0.25, ... , 0.95
def round_to_nearest_custom(x):
    return np.round((x - 0.05) / 0.1) * 0.1 + 0.05

df0['latitude_rounded'] = df0['latitude'].apply(round_to_nearest_custom)
df0['longitude_rounded'] = df0['longitude'].apply(round_to_nearest_custom)

df0.head(5)

,Unnamed: 0.1,Unnamed: 0,latitude,longitude,utc_time,xco2,baseXCO2,NE,local_time,year,...,leaf_area_index_low_vegetation,volumetric_soil_water_layer_1surface_net_solar_radiation_hourly,surface_net_thermal_radiation_hourly,volumetric_soil_water_layer_1,surface_net_solar_radiation_hourly,uv10m,LAI,emission,latitude_rounded,longitude_rounded
0,0,999250,34.164528,-118.48089,2023-02-06 18:00:11.725021952,421.83026,423.435196,-1.604935,2023-02-06 10:00:11.725021952-08:00,2023,...,1.024170,NaN,-367060.0,0.282516,1462010.0,2.731091,5.816528,NaN,34.15,-118.45
1,1,999251,34.170770,-118.49996,2023-02-06 18:00:11.752018432,421.61475,423.435949,-1.821203,2023-02-06 10:00:11.752018432-08:00,2023,...,1.024170,NaN,-367060.0,0.282516,1462010.0,2.731091,5.816528,NaN,34.15,-118.45
2,2,999252,34.140636,-118.34925,2023-02-06 18:00:11.896012800,421.81088,423.432313,-1.621430,2023-02-06 10:00:11.896012800-08:00,2023,...,1.668091,NaN,-379043.0,0.270538,1491254.0,2.343180,6.535889,NaN,34.15,-118.35
3,3,999253,34.147100,-118.36807,2023-02-06 18:00:11.923009280,421.96448,423.433093,-1.468615,2023-02-06 10:00:11.923009280-08:00,2023,...,0.794189,NaN,-376133.0,0.275803,1509558.0,2.847821,5.048218,NaN,34.15,-118.35
4,4,999254,34.153610,-118.38710,2023-02-06 18:00:11.950005504,422.16742,423.433878,-1.266459,2023-02-06 10:00:11.950005504-08:00,2023,...,0.993042,NaN,-374100.0,0.267105,1470882.0,2.783441,5.993530,NaN,34.15,-118.35


In [4]:
import pandas as pd
import os
import xarray as xr
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

# 将 DataFrame 转换日期
df0['datetime'] = pd.to_datetime(df0['utc_time'], errors='coerce')
df0['year'] = df0['datetime'].dt.year
df0['month'] = df0['datetime'].dt.month
grouped = df0.groupby(['year', 'month'])

# 处理每个组的函数
def process_group(year, month, group, base_dir, file_pattern):
    # 构建 NetCDF 文件路径
    file_path = os.path.join(base_dir, file_pattern.format(year=year, month=month))
    print(year, month)
    # 如果 NetCDF 文件存在，打开并处理
    if os.path.exists(file_path):
        ds = xr.open_dataset(file_path)
        # 将 NetCDF 数据集转换为 DataFrame
        df_nc = ds.to_dataframe().reset_index()

        # 遍历该组内的每一行数据
        for index, row in group.iterrows():
            # 过滤出目标日期的数据
            target_day = row['datetime'].strftime('%Y-%m-%d')
            df_nc_filtered = df_nc[df_nc['nday'] == target_day]

            # 使用 np.isclose() 根据 latitude 和 longitude 进行匹配
            match = df_nc_filtered[
                np.isclose(df_nc_filtered['latitude'], row['latitude_rounded'], atol=0.01) & 
                np.isclose(df_nc_filtered['longitude'], row['longitude_rounded'], atol=0.01)
            ]

            # 如果找到匹配的数据，提取 emission 并更新到 df0
            if not match.empty:
                df0.at[index, 'emission'] = match['emission'].values[0]  # 假设 emission 只有一个值
            else:
                print(f"未找到匹配数据：日期={target_day}, 纬度={row['latitude_rounded']}, 经度={row['longitude_rounded']}")
    else:
        print(f"文件不存在: {file_path}")

# 定义文件目录和文件模式
base_dir = r"E:\\GRACED_19082311"
file_pattern = "CarbonMonitor_total_y{year}_m{month:02d}.nc"

# 创建线程池
with ThreadPoolExecutor(max_workers=4) as executor:  # 根据你的 CPU 资源设置适当的线程数
    futures = []
    for (year, month), group in grouped:
        # 提交任务到线程池
        futures.append(executor.submit(process_group, year, month, group, base_dir, file_pattern))

    # 等待所有线程完成
    for future in as_completed(futures):
        future.result()  # 获取线程结果，确保任何异常被抛出

# 保存处理后的 df0
df0.to_csv(r"E:\\NE_glb_cities_era5_ae_1022_multi.csv", index=False)


2019 8
2019 9
2019 10
2019 11
2019 12
2020 1


D:\SOFTWARE\anaconda3\Lib\site-packages\xarray\backends\plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


2020 2
2020 3
2020 4
2020 5
2020 6
2020 7
2020 8
2020 9
2020 10
2020 11
2020 12
2021 1
2021 2
2021 3
2021 4
2021 5
2021 6
2021 7
2021 8
2021 9
2021 10
2021 11
2021 12
2022 1
2022 2
2022 3
2022 4
2022 5
2022 6
2022 7
2022 8
2022 9
2022 10
2022 11
2022 12
2023 1
2023 2
2023 3
2023 4
2023 5
2023 6
2023 7
2023 8
2023 9
2023 10
2023 11


MemoryError: Unable to allocate 371. MiB for an array with shape (194400000,) and data type int16

In [ ]:
df0

In [5]:
import os
import xarray as xr
import pandas as pd
import numpy as np

def get_emission_value(lat, lon, utc_time):
    # 获取年和月
    year = utc_time.year
    month = utc_time.month

    # 构造文件名的模式
    file_pattern = f"CarbonMonitor_total_y{year}_m{month:02d}.nc"
    file_path = os.path.join(r"E:\\GRACED_19082311", file_pattern)

    if os.path.exists(file_path):
        #print(f"Processing file: {file_path}")

        # 读取 GRACED 数据集
        dataset = xr.open_dataset(file_path, engine="netcdf4")

        # 提取必要的维度信息
        times = pd.to_datetime(dataset['nday'].values)

        # 找到最接近的日期对应的 nday
        nearest_time_idx = np.argmin(np.abs(times - utc_time))
        selected_nday = times[nearest_time_idx]
        #print(f"Selected nday: {selected_nday}")

        # 在该时间点提取数据
        dataset_slice = dataset.sel(nday=selected_nday)

        # 将目标经纬度四舍五入到最近的 y.x 格式
        rounded_lat = np.round(lat, 1)
        rounded_lon = np.round(lon, 1)
        #print(f"Rounded latitude: {rounded_lat}, Rounded longitude: {rounded_lon}")

        # 获取最接近的经纬度数据点
        latitudes = dataset_slice['latitude'].values
        longitudes = dataset_slice['longitude'].values

        # 找到最接近的经纬度
        nearest_lat_idx = np.abs(latitudes - rounded_lat).argmin()
        nearest_lon_idx = np.abs(longitudes - rounded_lon).argmin()

        nearest_lat = latitudes[nearest_lat_idx]
        nearest_lon = longitudes[nearest_lon_idx]

        #print(f"Nearest latitude in data: {nearest_lat}, Nearest longitude in data: {nearest_lon}")

        # 提取对应位置的数值 (假设变量名为 'emission')
        emission_value = dataset_slice['emission'].sel(latitude=nearest_lat, longitude=nearest_lon, method='nearest').values
        return emission_value
    else:
        print(f"File {file_pattern} not found.")
        return np.nan

# 示例 DataFrame
df0['utc_time'] = pd.to_datetime(df0['utc_time'])

# 计算 emission 值并添加到 DataFrame 中
df0['emission'] = df0.apply(lambda row: get_emission_value(row['latitude'], row['longitude'], row['utc_time']), axis=1)

df0.to_csv("E:\\NE_glb_cities_era5_ae_0822.csv")


In [26]:
import os
import xarray as xr
import pandas as pd
import numpy as np

# 假设你有一个经纬度和 utc_time 的输入
latitude = 34.059  # 示例纬度
longitude = -118.2567  # 示例经度
utc_time = pd.to_datetime('2020-02-06 10:00:00')  # 示例时间

# 获取年和月
year = utc_time.year
month = utc_time.month

# 构造文件名的模式 (根据文件命名规则)
file_pattern = f"CarbonMonitor_total_y{year}_m{month:02d}.nc"
file_path = os.path.join(r"E:\\GRACED_19082311", file_pattern)

if os.path.exists(file_path):
    print(f"Processing file: {file_path}")

    # 读取 GRACED 数据集
    dataset = xr.open_dataset(file_path, engine="netcdf4")

    # 提取必要的维度信息
    times = pd.to_datetime(dataset['nday'].values)

    # 找到最接近的日期对应的 nday
    nearest_time_idx = np.argmin(np.abs(times - utc_time))
    selected_nday = times[nearest_time_idx]
    print(f"Selected nday: {selected_nday}")

    # 在该时间点提取数据
    dataset_slice = dataset.sel(nday=selected_nday)

    # 将目标经纬度四舍五入到最近的 y.x 格式
    rounded_lat = np.round(latitude, 1)
    rounded_lon = np.round(longitude, 1)
    print(f"Rounded latitude: {rounded_lat}, Rounded longitude: {rounded_lon}")

    # 获取最接近的经纬度数据点
    latitudes = dataset_slice['latitude'].values
    longitudes = dataset_slice['longitude'].values

    # 找到最接近的经纬度
    nearest_lat_idx = np.abs(latitudes - rounded_lat).argmin()
    nearest_lon_idx = np.abs(longitudes - rounded_lon).argmin()

    nearest_lat = latitudes[nearest_lat_idx]
    nearest_lon = longitudes[nearest_lon_idx]

    print(f"Nearest latitude in data: {nearest_lat}, Nearest longitude in data: {nearest_lon}")

    # 提取对应位置的数值 (假设变量名为 'emission')
    emission_value = dataset_slice['emission'].sel(latitude=nearest_lat, longitude=nearest_lon, method='nearest').values
    print(f"Emission value at nearest point: {emission_value}")

else:
    print(f"File {file_pattern} not found.")


Processing file: E:\\GRACED_19082311\CarbonMonitor_total_y2020_m02.nc
Selected nday: 2020-02-06 00:00:00
Rounded latitude: 34.1, Rounded longitude: -118.3
Nearest latitude in data: 34.05000000000318, Nearest longitude in data: -118.2500000000035
Emission value at nearest point: 171934.140625


In [17]:
import xarray as xr
import pandas as pd

# 文件路径
file_path = r"E:\GRACED_19082311\CarbonMonitor_total_y2023_m02.nc"

# 打开NetCDF文件
ds = xr.open_dataset(file_path)

# 假设 'nday' 是时间格式（例如 datetime 或 pandas 的 Timestamp）
# 将数据集转换为DataFrame
df = ds.to_dataframe().reset_index()

# 过滤出 2019年09月05日 的数据，假设 'nday' 是包含日期信息的列
df_filtered = df[df['nday'] == pd.Timestamp('2023-02-06')]

# 查看过滤后的DataFrame
print(df_filtered)


           latitude  longitude       nday  emission
5             89.95    -179.95 2023-02-06       0.0
33            89.95    -179.85 2023-02-06       0.0
61            89.95    -179.75 2023-02-06       0.0
89            89.95    -179.65 2023-02-06       0.0
117           89.95    -179.55 2023-02-06       0.0
...             ...        ...        ...       ...
181439865    -89.95     179.55 2023-02-06       0.0
181439893    -89.95     179.65 2023-02-06       0.0
181439921    -89.95     179.75 2023-02-06       0.0
181439949    -89.95     179.85 2023-02-06       0.0
181439977    -89.95     179.95 2023-02-06       0.0

[6480000 rows x 4 columns]


In [9]:
df_filtered #这里是某一个ae数据

,latitude,longitude,nday,emission
4,89.95,-179.95,2019-09-05,0.0
34,89.95,-179.85,2019-09-05,0.0
64,89.95,-179.75,2019-09-05,0.0
94,89.95,-179.65,2019-09-05,0.0
124,89.95,-179.55,2019-09-05,0.0
...,...,...,...,...
194399854,-89.95,179.55,2019-09-05,0.0
194399884,-89.95,179.65,2019-09-05,0.0
194399914,-89.95,179.75,2019-09-05,0.0
194399944,-89.95,179.85,2019-09-05,0.0


In [41]:
print(df_filtered['latitude'].unique())
print(89.75 in df_filtered['latitude'].unique())
#但是这里我们发现明明89.75就在数据集里 为什么结果输出false呢？

[ 89.95  89.85  89.75 ... -89.75 -89.85 -89.95]
False


In [18]:
import numpy as np

# 使用 numpy.isclose() 对 latitude 和 longitude 进行筛选
df_filtered_selected = df_filtered[np.isclose(df_filtered['latitude'], 34.15) & np.isclose(df_filtered['longitude'], -118.45)]

# 查看筛选结果
print(df_filtered_selected)
#34.15            -118.45

          latitude  longitude       nday      emission
56263625     34.15    -118.45 2023-02-06  37833.453125


In [37]:
import numpy as np

# 检查 89.75 是否接近于 df_filtered['latitude'] 中的任意唯一值
is_present = np.isclose(89.75, df_filtered['latitude'].unique()).any()

print(is_present)  # 这将输出 True


True


In [34]:
df_filtered['latitude'].info()

<class 'pandas.core.series.Series'>
Index: 6480000 entries, 4 to 194399974
Series name: latitude
Non-Null Count    Dtype  
--------------    -----  
6480000 non-null  float64
dtypes: float64(1)
memory usage: 98.9 MB
